# Introduction to Artificial Intelligence
Laboratory 1



Exercise 1 Route searching 
1.	Create a set of cities (as points) with coordinates x, y on a plane with height as z coordinate. The cost of going from city A to city B is equal to the Euclidean distance between two cities, if there exists a road. You should define scenarios according to two criteria: 
a.	There are all the direct connections / c.a. 80% of possible connections
b.	The problem is symmetrical / asymmetrical (in asymmetrical – going up is height +10%, going down: -10%)
You should choose the coordinates randomly from the range <-100, 100> for x,y and <0, 50> for z.
2.	Represent the created map as a weighted (directed) graph, where cities are the nodes and roads are the edges of the graph.
3.	In the created scene, solve the traveling salesman problem: The salesman starts from a chosen city and has to visit every city exactly once before returning to the starting city. The goal is to find a path with the lowest cost.
In the problem, we define state as a partial or full path from the starting city and the corresponding state. You should represent the search problem in a form of state tree.
a.	Implement a full search of the tree, using BFS and DFS methods.
b.	Approximate the solution using greedy search (NN and Dijkstra)
c.	Solve/approximate the solution using A* with inadmissible/admissible heuristics
d.	Approximate the solution using ACO algorithm
4.	Test each algorithm, in each scenario, for n=5…20 cities, in terms of the found path cost, time and memory consumption.


In [2]:
import random
from support import euclidean_distance,euclidean_distance_assymetrical
class CitiesMap():
    def __init__(self,n,seed = None):
        """
        initialize with number of cities to include in graph 
        """    
        
        if seed is not None:
            random.seed(seed)
        self.seed = seed
        self.n = n 
        self.cities = []
        self.routes = [[0 for _ in range(n)] for _ in range(n)]
        # we can represent routes between cities as a matrix with n x n shape 
        for i in range(n):
            self.cities.append((random.randint(-101,101),random.randint(-101,101),random.randint(0,51)))

        # now let's make sure that we don't have duplicated cities so we'll get derired number of unique cities
        
        len_check = len(set(self.cities))
        
        while (len_check != n):
            self.cities.append((random.randint(-101,101),random.randint(-101,101),random.randint(0,51)))
            len_check = len(set(self.cities))
        self.cities = list(set(self.cities))
    
    
    def calculate_distance(self,symmetrical: bool = True , connections: float = 1):
        """
        Fill matrix with distnaces between city i and j  0 < i,j < n with 0 on main diagonal , conenctions 1 means all cities will be connected
        0.5 only half of them and so on 
        
        """
             
        if (symmetrical):
            if (connections==1):   
                for i in range(self.n):
                    for j in range(i+1 , self.n):  # to not double calcualte
                        
                        distance = euclidean_distance(self.cities[i], self.cities[j])
                    
                        # Store the route only once, as it is undirected
                        self.routes[i][j] = distance
                        self.routes[j][i] = distance
                return self.routes
            else:
                # in this case we nned to delete number of connections that equals to connections * (n^2-n)/2 
                
                n_to_delete = int(round((1 - connections) * ((self.n**2 - self.n) / 2),0))

                indexes_list = []    
                                
                for i in range(self.n):
                    for j in range(i+1 , self.n):  # to not double calcualte
                        
                        indexes_list.append((i,j))
                        
                        distance = euclidean_distance(self.cities[i], self.cities[j])
                    
                        # Store the route only once, as it is undirected
                        self.routes[i][j] = distance
                        self.routes[j][i] = distance
            
                indexes_to_delete = random.sample(indexes_list,n_to_delete)
                
                for i in indexes_to_delete:
                    # in the list that we iterate over we have pair of matrix indexes when let's say i means position with 0 index in matrix rows, j column
                    self.routes[i[0]][i[1]] = 0     # deletes i,i
                    self.routes[i[1]][i[0]] = 0        # deletes j,i 
                    
                    
                
                
                return self.routes
                
                
                
        else: 
            if (connections==1):  
                for i in range(self.n):
                    for j in range(self.n):
                        self.routes[i][j] = euclidean_distance_assymetrical(self.cities[i], self.cities[j])
                        
                return self.routes  
            else:
                # in this case we nned to delete number of connections that equals to connections * (n^2-n) # assymetrical 
                
                n_to_delete = int(round((1 - connections) * ((self.n**2 - self.n) / 2),0))
                indexes_list = []    
                                
                for i in range(self.n):
                    for j in range(self.n):
                        # however with connections to delete we'll keep approach from previous one as I understand that lack connections means no road between two points 
                        # no matter how long 
                        
                        self.routes[i][j] = euclidean_distance_assymetrical(self.cities[i], self.cities[j])
                        
                        # adds to indexes base only ones from above main diagonal 
                        if (j>i):
                            indexes_list.append((i,j))
                    
                    
                        
                indexes_to_delete = random.sample(indexes_list,n_to_delete)
                
                
                for i in indexes_to_delete:
                    # in the list that we iterate over we have pair of matrix indexes 
                    self.routes[i[0]][i[1]] = 0     # deletes i,i
                    self.routes[i[1]][i[0]] = 0
                    
                    
                
                
                return self.routes
                
                
    
    


In [20]:
from queue import PriorityQueue
from sys import maxsize
class RouteSearch:
    def __init__(self,cities_map : CitiesMap):
        self.nodes = cities_map.cities
        self.size = cities_map.n
        self.edges = cities_map.routes
        # self.states = []
        self.max = maxsize
    def dfs_all_paths(self, starting_node: int):
        """
        Perform DFS to explore all possible paths, both complete and incomplete.
        
        Parameters:
        starting_node : int - the node from which to start the search
        
        Returns:
        list : A list of paths. Path can be incomplete or complete.
        """
        # Stack for DFS
        stack = [[starting_node]]
        all_paths = []
        step = 0
        while stack:
            path = stack.pop()
            last_node = path[-1]
            
            
            # Record the current (incomplete or complete) path and its cost
            all_paths.append(path)

            # If the path includes all cities, check if it can return to the start
            if len(path) == self.size:
                return_to_start_cost = self.edges[last_node][starting_node]
                if return_to_start_cost != 0:     # to make sure that we can return to start from there
                    # Record the completed cycle (path returns to the start)
                    all_paths.append(path + [starting_node])

            # Explore all unvisited neighbors (continue DFS)
            for neighbor in range(self.size - 1, -1, -1):  # reverse order to simulate stack behavior
                if neighbor not in path and self.edges[last_node][neighbor] != 0:
                    new_path = path + [neighbor]
                    stack.append(new_path)
                    
        return all_paths  

    def bfs_all_paths(self, starting_node: int):
        """
        Perform BS to explore all possible paths, both complete and incomplete.
        
        Parameters:
        starting_node : int - the node from which to start the search
        
        Returns:
        list : A list of paths. Path can be incomplete or complete.
        """
        from collections import deque

        # queue for bfs
        # queue = [[starting_node]]  # with this by hand implementation this algorithm was useless already around n = 10. As I read due to O(n) pop(0) opearation. 
        queue = deque([[starting_node]]) 

        all_paths = []

        while queue:
            # path = queue.pop(0)   # take first element from list (queue)
            path = queue.popleft()
            last_node = path[-1]
            
            # Record the current (incomplete or complete) path and its cost
            all_paths.append(path)

            # If the path includes all cities, check if it can return to the start
            if len(path) == self.size:
                return_to_start_cost = self.edges[last_node][starting_node]   # to make sure that we can return to start from there
                if return_to_start_cost != 0:
                    # Record the completed cycle (path returns to the start)
                    all_paths.append(path + [starting_node])

                continue

            # Explore all unvisited neighbors (continue bfs)
            for neighbor in range(0, self.size):  # normal queue order
                if neighbor not in path and self.edges[last_node][neighbor] != 0:
                    new_path = path + [neighbor]
                    queue.append(new_path)

        return all_paths
    
    def get_distance(self,cycle):
        distance = 0
        for index in range(self.size):  # it's in fact len(state_tree) - 1 -> exactly what we need in this approach
            distance += self.edges[cycle[index]][cycle[index+1]]  # we need to extract ith and ith+1 element of our cycle 
        return distance
    
    def get_min_distance(self,state_tree):
        """ Requires state tree object from for example dfs or bfs algo, calculate distance only for full paths"""
        
        min_distance = self.max
        shortest_route = []
        for cycle in state_tree:
            if (len(cycle)==self.size + 1):  # we calculate distance only in case of full cycle
                distance = self.get_distance(cycle)
            
                if (distance < min_distance):
                    min_distance = distance
                    shortest_route = cycle

        return (shortest_route,min_distance)
    
    def nn(self,start):
        path = [start]
        
        while(len(path)!=self.size):
            dist = maxsize  # initialize distance as maxsize
            node = path[-1]  # get last node from path as a new 'starting node'
            target = -1  # initialize target as a number from outside matrix index
            for neighbour in range(self.size):   # iterate over neighbours of 'new' target                 
                cur_dist = self.edges[node][neighbour] 
                if ((neighbour not in path) & (0 < cur_dist < dist)):    # if path exists and is shorter than previous lower from this starting point 
                    target = neighbour   # store neighbour
                    dist = cur_dist   # store shortest distance 
                    
            if target == -1:    # we got in the dead end 
                return path , "Didn't find complete route"
            path.append(target)
        
        if (self.edges[target][start]!=0):    # if we reached there and we can go back to start we found complete circle
            path.append(start)
            return path , self.get_distance(path)
        
        # póki co nie testuję nn
        return path, "Didn't find complete route - no connection between last node and starting node"
    
    
    
    def a_star(self,start,heuristic):
        
        pq = PriorityQueue() 
        
        pq.put((0, [start])) # pair, distance path 
        
         # Track best solution
        best_tour = None
        best_distance = maxsize
        while not pq.empty():
            distance , path = pq.get()
            current_node = path[-1]
            if len(path) == self.size:
                if self.edges[current_node][start] != 0:
                    total_distance = distance + self.edges[current_node][start]
                    complete_path = path + [start]
                   
                    # Update best tour
                    if total_distance < best_distance:
                        best_tour = complete_path
                        best_distance = total_distance
                        # if best_distance == minimum:
                        #     return best_tour, best_distance

                continue
            
            for neighbour in range(self.size):
                if neighbour not in path and self.edges[current_node][neighbour] != 0:                         
                    new_distance = distance + self.edges[current_node][neighbour]
                    
                    # Create new path
                    new_path = path + [neighbour]
                    if heuristic:
                        h = heuristic(new_path, self)
                        new_distance += h
                    
                    pq.put((new_distance,new_path))
            

            
        return best_tour, best_distance
    
    def dijkstra(self,start):
        return self.a_star(start,heuristic=None)

    def aco(self, alfa =0.9 , beta = 1.5 , decay = .5, Q = 5,steps = 10, ants = None ,best_half_update= True,random_seed = 42):
        if random_seed:
            random.seed(random_seed)
        
        # default to number of cities in the problem
        if ants:
            pass
        else:  
            ants = self.size    
        
        # initialize matrix for pheromones storage 
        phero_table = [[1 for _ in range(self.size)] for _ in range(self.size)]
        shortest_distance = maxsize
        def ant_path(start):
            path = [start]
            path_distance = 0
            
            # we will first create list of possibles continuations and their probs 
            while len(path) < self.size: 
                current = path[-1]

                probs = []
                possible_steps = []
                for neighbour in range(self.size):
                    dist = self.edges[current][neighbour]
                    if ((dist != 0)& (neighbour not in path)):
                        possible_steps.append(neighbour)  # we append possible step together with pheromone of it 
                        prob_temp = phero_table[current][neighbour]**alfa / dist **beta # we store distance there 
                        probs.append(prob_temp)

                if len(possible_steps) == 0: 
                    # print("First if fired")
                    return path,maxsize   # we went to dead end 
                next_step = random.choices(possible_steps,weights=probs)[0]    # we are choosing next step, appending distance and direction 
                path_distance += self.edges[current][next_step]
                path.append(next_step)
                current = next_step
            # try to return to start 
            if (self.edges[current][path[0]]!=0):
                path.append(start)
                path_distance += self.edges[current][path[0]] # back to the start 
            else:
                return path, maxsize  # we will divide by very huge number so we will update pheromon by very low value  
            # if we get there we found complet cycle. Let's return path and  distnace of cycle
            return path, path_distance
            
        
        for step in range(steps):
            # we will place ants in order and if ants>self.size then we will start from the 0 index city again 
            results_table = []
            
            for ant in range(ants):
                results_table.append(ant_path(ant % self.size))

            # also let's keep best solution in step for now just for printing it out
            results_table.sort(key = lambda x: x[1])

            # if update only based on top half of solution 
            if best_half_update:
                results_table = results_table[:len(results_table)//2]
            
            
            # now we need to update phero_table according to formula, first of all we can apply decay rate 
            
            for i in range(self.size):
                for j in range(self.size):
                    phero_table[i][j] = phero_table[i][j] * (1 - decay)
        
            # now update based on result table - solutions generated by ants in previous step 
            for route, distance in results_table: 
                for k in range (len(route)-1):
                    r = route[k]
                    s = route[k+1]   # we extract pairs of city on path of this ant
                    # and update pehero_table in cosent with formula
                    phero_table[r][s] += Q / distance                         
            print(f"Step: {step} best route distance: {results_table[0][1]}")
        
            # keep best soluton 
            if results_table[0][1] < shortest_distance:
                shortest_distance = results_table[0][1]
                best_solution = results_table[0][0] , results_table[0][1]
        
        
        
        return best_solution

In [21]:
# test = CitiesMap(20,seed=42)  # this will generate route without connection with nn
test = CitiesMap(8,seed=40)
test.calculate_distance(symmetrical=False,connections=1);

In [22]:
algos = RouteSearch(test)

In [23]:

paths = algos.dfs_all_paths(0)


In [24]:
paths1 = algos.bfs_all_paths(0)

In [25]:
algos.get_min_distance(paths)   # based on route from dfs or bfs 

([0, 3, 1, 2, 7, 5, 4, 6, 0], 597.1694282321282)

In [26]:
algos.nn(0)

([0, 3, 1, 2, 7, 5, 6, 4, 0], 644.5543746618329)

In [27]:
algos.dijkstra(0)

([0, 3, 1, 2, 7, 5, 4, 6, 0], 597.1694282321282)

In [10]:
# algos.dijkstra_nn(0)

In [19]:
algos.a_star(0)

TypeError: RouteSearch.a_star() missing 1 required positional argument: 'heuristic'

In [7]:
algos.aco(steps=10,decay=0.9,Q=10,best_half_update=True,random_seed=42)

Step: 0 best route distance: 1322.1719625519872
Step: 1 best route distance: 1306.7340745011898
Step: 2 best route distance: 1338.5234264132598
Step: 3 best route distance: 1100.2351982563302
Step: 4 best route distance: 1109.8646026382376
Step: 5 best route distance: 1012.8692123805015
Step: 6 best route distance: 979.1089172104698
Step: 7 best route distance: 988.9428090830841
Step: 8 best route distance: 979.2053171941974
Step: 9 best route distance: 950.8846739249034


([0, 6, 3, 10, 13, 19, 17, 8, 5, 11, 14, 15, 4, 1, 16, 7, 9, 18, 2, 12, 0],
 950.8846739249034)